In [1]:
"""
@author: Prakhar
"""
import os
import argparse
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
torch.cuda.empty_cache()

def choose_from_top_k_top_n(probs, k=50, p=0.8):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    top_prob = {i: top_prob[idx] for idx, i in enumerate(ind)}
    sorted_top_prob = {k: v for k, v in sorted(
        top_prob.items(), key=lambda item: item[1], reverse=True)}

    t = 0
    f = []
    pr = []
    for k, v in sorted_top_prob.items():
        t += v
        f.append(k)
        pr.append(v)
        if t >= p:
            break
    top_prob = pr / np.sum(pr)
    token_id = np.random.choice(f, 1, p=top_prob)

    return int(token_id)


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device_str = "cuda" if torch.cuda.is_available() else "cpu"

device = torch.device("cpu")
device_str = "cpu"


def generate(tokenizer, model, sentences, label):
    with torch.no_grad():
        for idx in range(sentences):
            finished = False
            cur_ids = torch.tensor(tokenizer.encode(
                label)).unsqueeze(0).to(device_str)
                
            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]

                softmax_logits = torch.softmax(logits[0, -1], dim=0)

                if i < 5:
                    n = 10
                else:
                    n = 5

                next_token_id = choose_from_top_k_top_n(softmax_logits.to(device_str).numpy())  # top-k-top-n sampling
                cur_ids = torch.cat([cur_ids, torch.ones((1, 1)).long().to(device) * next_token_id], dim=1)
                

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    finished = True
                    break

                

            if finished:
                output_list = list(cur_ids.squeeze().to(device_str).numpy())
                output_text = tokenizer.decode(output_list)
                print(output_text)
            else:
                output_list = list(cur_ids.squeeze().to(device_str).numpy())
                output_text = tokenizer.decode(output_list)
                print(output_text)




def load_models(model_name):

    os.environ['CUDA_VISIBLE_DEVICES'] ='0'
    
    """
    Summary:
            Loading the trained model
    """
    print('Loading Trained GPT-2 Model')
    tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
    model = GPT2LMHeadModel.from_pretrained('distilgpt2')
    model_path = model_name
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device_str)))
    return tokenizer, model








In [2]:
SENTENCES = 5
MODEL_NAME = 'mymodel.pt'
# MODEL_NAME = 'gpt2_medium_spamham_4.pt'
LABEL = 'SPAM'

TOKENIZER, MODEL = load_models(MODEL_NAME)

generate(TOKENIZER, MODEL, SENTENCES, LABEL)


Loading Trained GPT-2 Model
SPAM: Thanks for the chance to interview me <|endoftext|>
SPAM: Japkala is here. Do you have any questions? KABATYN: Thanks for visiting. Send me a call on +51 4727383887 to get an email from japkala <|endoftext|>
SPAM: Sorry, if you are going to a pub you want to get 2. Call ahead on your phone for FREE. For details please call our Phone Call 072-795566 or call 07544666940. We are also open 5th street so call if there is a busy lunchtime, you can call <|endoftext|>
SPAM: We are trying to make sure that you have a quick and enjoyable start to what we want to do with your future. We have been looking at a few things which will help you make your life easier. <|endoftext|>
SPAM: You have a new phone app for your mobile phone <|endoftext|>


In [3]:
SENTENCES = 5
# MODEL_NAME = 'mymodel.pt'
MODEL_NAME = 'gpt2_medium_spamham_4.pt'
LABEL = 'HAM'




TOKENIZER, MODEL = load_models(MODEL_NAME)

generate(TOKENIZER, MODEL, SENTENCES, LABEL)


Loading Trained GPT-2 Model


RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
	Missing key(s) in state_dict: "transformer.h.0.attn.masked_bias", "transformer.h.1.attn.masked_bias", "transformer.h.2.attn.masked_bias", "transformer.h.3.attn.masked_bias", "transformer.h.4.attn.masked_bias", "transformer.h.5.attn.masked_bias". 
	Unexpected key(s) in state_dict: "transformer.h.6.ln_1.weight", "transformer.h.6.ln_1.bias", "transformer.h.6.attn.bias", "transformer.h.6.attn.c_attn.weight", "transformer.h.6.attn.c_attn.bias", "transformer.h.6.attn.c_proj.weight", "transformer.h.6.attn.c_proj.bias", "transformer.h.6.ln_2.weight", "transformer.h.6.ln_2.bias", "transformer.h.6.mlp.c_fc.weight", "transformer.h.6.mlp.c_fc.bias", "transformer.h.6.mlp.c_proj.weight", "transformer.h.6.mlp.c_proj.bias", "transformer.h.7.ln_1.weight", "transformer.h.7.ln_1.bias", "transformer.h.7.attn.bias", "transformer.h.7.attn.c_attn.weight", "transformer.h.7.attn.c_attn.bias", "transformer.h.7.attn.c_proj.weight", "transformer.h